In [2]:
pip install -qU langchain-community "langchain-chroma>=0.1.2" langchain-huggingface langchain-groq langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders  import TextLoader
loader = TextLoader('D:\Finsafe\Finalfin.txt')
pages = loader.load()

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

HF_TOKEN = "api_key"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\G_dha\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


c:\Users\G_dha\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from langchain.vectorstores import Chroma

persist_directory = 'chroma_DB'

vectordb = Chroma.from_documents(splits, embeddings, persist_directory=persist_directory)
vectordb.persist()
vectordb


c:\Users\G_dha\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [11]:
docs = vectordb.similarity_search('who are the team members names?')

In [12]:
docs[0].page_content

'Anusha Rathore, has more than 20 years experience spanning across banking & Mutual fund industry.\nAfter working for over decade, handling multiple functions like managing services, sales and sales support at Citicorp and HDFC Asset Management Company .She joined Finsafe, a finance education organisation in 2015, which is focused on equipping Indians with strategies to handle their money based on goals, growth, and safety. She is a senior consultant and financial educator and currently handles training and content development for Finsafe. Anusha has trained over 1,00,000 associates.\nAnusha with her extensive knowledge in investment products, loans, insurance and tax planning has put together content for the e-learning programs developed at Finsafe. She was also part of Womantra, a program developed exclusively for working women professionals.\nSoumya Anand, comes with more than 18 years experience spanning across Mutual fund and financial services industry.'

In [13]:
retriever = vectordb.as_retriever()


In [15]:
import os

os.environ["GROQ_API_KEY"] = "api key"

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")


In [30]:
llm.predict('what is u r name?')

c:\Users\G_dha\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


"I don't have a personal name. I'm an AI designed to assist and communicate with users like you, so I don't have a personal identity or a physical presence. I exist solely to help answer your questions and provide information to the best of my ability."

In [66]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


template = """You are an intelligent and concise assistant for  Finsafe organization.
 Use the provided context to answer the user's question clearly and accurately in an interactive manner. 
 If you don't know the answer, simply state that you don't know—do not fabricate a response. 
 Keep your answers to no more than three sentences and always conclude with, 'Thanks for asking!' 
 Additionally, include this statement: 'For more inquiries, contact +91 7411677575 or reach out via email at support@finsafe.in.'"

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)
def format_docs(pages):
    return "\n\n".join(doc.page_content for doc in pages)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [69]:
rag_chain.invoke("what is the price of each course?")

'The charges for Finsmart are Rs. 4,000 + taxes. Finstart is priced at Rs. 1,500 + taxes.'